In [1]:
# Import necessary packages
using PyPlot
using SparseArrays
using LinearAlgebra
using IterativeSolvers
using AlgebraicMultigrid
using Printf

In [29]:
struct Grid
    x::Vector
    y::Vector
    xc::Vector
    yc::Vector
    nx::Int
    ny::Int
    W::Float64
    H::Float64
    dx::Float64
    dy::Float64
    function Grid(W::Float64,H::Float64,nx::Int,ny::Int)
        dx = W/(nx-1)
        dy = H/(ny-1)
        new(LinRange(0,W,nx),LinRange(0,H,ny),
            LinRange(0-dx/2,W+dx/2,nx+1),LinRange(0-dy/2,H+dy/2,ny+1),
            nx,ny,W,H,W/(nx-1),H/(ny-1))
    end
end

struct BoundaryConditions
    # The intent here is that each boundary gets a flag
    # 0 = Free-slip
    # 1 = No-slip
    # other possibilities?
    top::Int
    bottom::Int
    left::Int
    right::Int
end

function initial_conditions(grid::Grid)
    # Setup the initial density structure
    rho = zeros(grid.ny,grid.nx)
    etaS = ones(grid.ny,grid.nx)
    etaN = ones(grid.ny,grid.nx)
    for i in 1:grid.ny
        for j in 1:grid.nx
            if grid.x[j] < grid.W/2.
                rho[i,j] = 3200.
                etaS[i,j] = 1.0e21
            else
                rho[i,j] = 3300.
                etaS[i,j] = 1.0e21
            end
            if grid.xc[j] < grid.W/2.
                etaN[i,j] = 1.0e21
            else
                etaN[i,j] = 1.0e21
            end
        end
    end
    
    return rho, etaS, etaN
end

node_index(i::Int,j::Int,ny::Int) = ny*(j-1)+i
vxdof(i::Int,j::Int,ny::Int) = 3*(node_index(i,j,ny)-1)+1
vydof(i::Int,j::Int,ny::Int) = 3*(node_index(i,j,ny)-1)+2
pdof( i::Int,j::Int,ny::Int) = 3*(node_index(i,j,ny)-1)+3


function form_stokes(grid::Grid,eta_s::Matrix,eta_n::Matrix,bc::BoundaryConditions,gx::Float64,gy::Float64)
    k::Int = 1 # index into dof arrays
    nx = grid.nx
    ny = grid.ny
    nn = nx*ny
    nnz = 2*11*nn + 4*nn # total number of nonzeros in matrix (not including BCs)
    row_index = zeros(Int64,nnz) # up to 5 nonzeros per row
    col_index = zeros(Int64,nnz) 
    value = zeros(Float64, nnz)
    kcont = 2*eta_s[1,1]/(grid.dx+grid.dy)# scaling factor for continuity equation
    kbond = 1.# scaling factor for dirichlet bc equations.

    R=zeros(3*nn,1)
    
    # loop over j
    for j in 1:nx
        # loop over i
        for i in 1:ny
            dxp = j<nx ? grid.x[j+1] - grid.x[j]   : grid.x[j]   - grid.x[j-1]
            dxm = j>1  ? grid.x[j]   - grid.x[j-1] : grid.x[j+1] - grid.x[j]
            dxc = 0.5*(dxp+dxm)
            dyp = i<ny ? grid.yc[i+1]-grid.yc[i]   : grid.yc[i]   -grid.yc[i-1]
            dym = i>1  ? grid.yc[i] - grid.yc[i-1] : grid.yc[i+1] -grid.yc[i]
            dyc = 0.5*(dyp+dym)

            # discretize the x-stokes - note that numbering in comments refers to Gerya Figure 7.18a
            # and equation 7.22
            this_row = vxdof(i,j,ny)
            # Boundary cases first...            
            if j==1 || j == nx # left boundary or right boundary
                # vx = 0
                row_index[k] = this_row
                col_index[k] = this_row
                value[k] = kbond
                k+=1
                R[this_row] = 0.0 *kbond
            elseif i==1
                # dvx/dy = 0 (free slip)
                row_index[k] = this_row
                col_index[k] = this_row
                value[k] = -kbond
                k+=1
                row_index[k] = this_row
                col_index[k] = vxdof(i+1,j,ny)
                value[k] = kbond
                k+=1                
                R[this_row] = 0.0*kbond
            else
            # vx1
            row_index[k] = this_row
            col_index[k] = vxdof(i,j-1,ny)
            value[k] = 2*eta_n[i,j]/dxm/dxc
            k+=1
            # vx2
            row_index[k] = this_row
            col_index[k] = vxdof(i-1,j,ny)
            value[k] = eta_s[i-1,j]/dym/dyc
            k+=1
            # vx3
            row_index[k] = this_row
            col_index[k] = this_row
            value[k] = -2*eta_n[i,j+1]/dxp/dxc -2*eta_n[i,j]/dxm/dxc - eta_s[i,j]/dyp/dyc - eta_s[i-1,j]/dym/dyc
            if i == ny #vx4
                # if i == nx, dvx/dy = 0 -> vx3 == vx4 (see Gerya fig 7.18a)
                value[k] += eta_s[i,j]/dyp/dyc
                k+=1
            else
                k+=1
                # vx4
            
                # enforce dvx/dy = 0 (free slip)                
                row_index[k] = this_row
                col_index[k] = vxdof(i+1,j,ny)
                value[k] = eta_s[i,j]/dyp/dyc
                k+=1
            end
                    
            # vx5
            row_index[k] = this_row
            col_index[k] = vxdof(i,j+1,ny)
            value[k] = 2*eta_n[i,j+1]/dxp/dxc
            k+=1
            # vy1
            row_index[k] = this_row
            col_index[k] = vydof(i-1,j,ny)
            value[k] = eta_s[i-1,j]/dxc/dyc
            k+=1
            # vy2
            row_index[k] = this_row
            col_index[k] = vydof(i,j,ny)
            value[k] = -eta_s[i,j]/dxc/dyc
            k+=1
            # vy3
            row_index[k] = this_row
            col_index[k] = vydof(i-1,j+1,ny)
            value[k] = -eta_s[i-1,j]/dxc/dyc
            k+=1
            # vy4
            row_index[k] = this_row
            col_index[k] = vydof(i,j+1,ny)
            value[k] = eta_s[i,j]/dxc/dyc           
            k+=1
            # P1
            row_index[k] = this_row
            col_index[k] = pdof(i,j,ny)
            value[k] = kcont/dxc
            k+=1
            # P2
            row_index[k] = this_row
            col_index[k] = pdof(i,j+1,ny)
            value[k] = -kcont/dxc
            k+=1
            
            R[this_row] = -gx*(rho[i-1,j]+rho[i,j])/2.
            end
            # END X-STOKES
            
            # BEGIN Y-STOKES
            dxp = j < nx ? grid.xc[j+1] - grid.xc[j]   : grid.xc[j]  -grid.xc[j-1]
            dxm = j > 1  ? grid.xc[j]   - grid.xc[j-1] : grid.xc[j+1]-grid.xc[j]
            dxc = j > 1  ? grid.x[j]    - grid.x[j-1]  : grid.x[j+1] - grid.x[j]
            dyp = i < ny ? grid.y[i+1]  - grid.y[i]    : grid.y[i]   - grid.y[i-1]
            dym = i > 1  ? grid.y[i]    - grid.y[i-1]  : grid.y[i+1] - grid.y[i]
            dyc = i < ny ? grid.yc[i+1] - grid.yc[i]   : grid.yc[i]  - grid.yc[i-1]            
            
            this_row = vydof(i,j,ny)
            if i==1 || i == ny
                # top row / bottom row
                row_index[k] = this_row
                col_index[k] = this_row
                value[k] = kbond
                k+=1
                R[this_row] = 0.0*kbond
            elseif j==1
                # left boundary - free slip
                row_index[k] = this_row
                col_index[k] = this_row
                value[k] = kbond
                k+=1
                row_index[k] = this_row
                col_index[k] = vydof(i,j+1,ny)
                value[k] = -kbond
                k+=1
                R[this_row] = 0.0*kbond
            else            
                #vy1
                row_index[k] = this_row
                col_index[k] = vydof(i,j-1,ny)
                value[k] = eta_s[i,j-1]/dxm/dxc
                k+=1
                #vy2
                row_index[k] = this_row
                col_index[k] = vydof(i-1,j,ny)
                value[k] = 2*eta_n[i,j]/dym/dyc
                k+=1
                #vy3
                row_index[k] = this_row
                col_index[k] = this_row
                value[k] = -2*eta_n[i+1,j]/dyp/dyc -2*eta_n[i,j]/dym/dyc - eta_s[i,j]/dxp/dxc - eta_s[i,j-1]/dxm/dxc
                if j == nx
                   # free slip - vx5 = vx3.
                   value[k] += eta_s[i,j]/dxp/dxc
                end
                k+=1
                
                #vy4
                row_index[k] = this_row
                col_index[k] = vydof(i+1,j,ny)
                value[k] = 2*eta_n[i+1,j]/dyp/dyc
                k+=1
                #vy5
                if j<nx
                    row_index[k] = this_row
                    col_index[k] = vydof(i,j+1,ny)
                    value[k] = eta_s[i,j]/dxp/dxc
                    k+=1
                end
                #vx1
                row_index[k] = this_row
                col_index[k] = vxdof(i,j-1,ny)
                value[k] = eta_s[i,j-1]/dxc/dyc
                k+=1
                #vx2
                row_index[k] = this_row
                col_index[k] = vxdof(i+1,j,ny)
                value[k] = -eta_s[i,j-1]/dxc/dyc
                k+=1
                #vx3
                row_index[k] = this_row
                col_index[k] = vxdof(i,j,ny)
                value[k] = -eta_s[i,j]/dxc/dyc
                k+=1
                #vx4
                row_index[k] = this_row
                col_index[k] = vxdof(i+1,j,ny)
                value[k] = eta_s[i,j]/dxc/dyc
                k+=1
                #P1
                row_index[k] = this_row
                col_index[k] = pdof(i,j,ny)
                value[k] = kcont/dyc
                k+=1
                #P2
                row_index[k] = this_row
                col_index[k] = pdof(i+1,j,ny)
                value[k] = -kcont/dyc
                k+=1

                R[this_row] = -gy*(rho[i,j-1]+rho[i,j])/2.
            end
            # END Y-STOKES
            
            # discretize the continuity equation
            # dvx/dx + dvy/dy = 0
            this_row = pdof(i,j,ny)
            if i==1 || j == 1 || (i==2 && j == 2)
                row_index[k] = this_row
                col_index[k] = this_row
                value[k] =  kbond
                k+=1  
                R[this_row] = 0.0
            else
                dxm = grid.x[j] - grid.x[j-1]
                dym = grid.y[i] - grid.y[i-1]
                
                row_index[k] = this_row
                col_index[k] = vxdof(i,j,ny)
                value[k] =  kcont/dxm
                k+=1

                row_index[k] = this_row
                col_index[k] = vxdof(i,j-1,ny)
                value[k] = -kcont/dxm
                k+=1

                row_index[k] = this_row
                col_index[k] = vydof(i,j,ny)
                value[k] =  kcont/dym
                k+=1

                row_index[k] = this_row
                col_index[k] = vydof(i-1,j,ny)
                value[k] = -kcont/dym
                k+=1

                R[this_row] = 0.0
            end
            # END CONTINUITY
            
        end
    end
    @views row_index = row_index[1:(k-1)]
    @views col_index = col_index[1:(k-1)]
    @views value = value[1:(k-1)]

    L = sparse(row_index,col_index,value)
    return L,R    
end

function unpack(solution, grid::Grid)
    P = zeros(Float64,(grid.ny,grid.nx))
    vx = zeros(Float64,(grid.ny,grid.nx))
    vy = zeros(Float64,(grid.ny,grid.nx))
    ny = grid.ny
     for j in 1:grid.nx
        for i in 1:grid.ny                
            vx[i,j] = solution[vxdof(i,j,grid.ny)]
            vy[i,j] = solution[vydof(i,j,grid.ny)]
            P[i,j]  = solution[pdof(i,j,grid.ny)]            
        end
    end
    return vx,vy,P
end

unpack (generic function with 1 method)

In [30]:
# Set up the grid
nx = 4
ny = 3
W = 1e6
H = 1.0e6
gy = -10.0
gx = -10. * 0.
grid = Grid(W,H,nx,ny)
rho,eta_s,eta_n = initial_conditions(grid)
bc = BoundaryConditions(0,0,0,0)
L,R = form_stokes(grid,eta_s,eta_n,bc,gx,gy)
solution = L\R
vx,vy,P = unpack(solution,grid)

([0.0 -9.230769230769257e-9 -9.230769230769307e-9 0.0; 0.0 -9.230769230769257e-9 -9.230769230769307e-9 0.0; 0.0 9.230769230769255e-9 9.230769230769307e-9 0.0], [0.0 0.0 0.0 0.0; 1.3846153846153884e-8 1.3846153846153884e-8 7.62541592871481e-23 -1.3846153846153958e-8; 0.0 0.0 0.0 0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 2.179487179487179e-8 4.3589743589743814e-8; 0.0 -6.7272435897435875e-6 -6.7490384615384594e-6 -6.770833333333332e-6])

In [31]:

for i in 1:grid.nx*grid.ny
    println(i," ",L[3*(i-1)+3,:])
end

1   [3 ]  =  1.0
2   [6 ]  =  1.0
3   [9 ]  =  1.0
4   [12]  =  1.0
5   [15]  =  1.0
6   [7 ]  =  -7.2e9
  [14]  =  -4.8e9
  [16]  =  7.2e9
  [17]  =  4.8e9
7   [21]  =  1.0
8   [13]  =  -7.2e9
  [20]  =  -4.8e9
  [22]  =  7.2e9
  [23]  =  4.8e9
9   [16]  =  -7.2e9
  [23]  =  -4.8e9
  [25]  =  7.2e9
  [26]  =  4.8e9
10   [30]  =  1.0
11   [22]  =  -7.2e9
  [29]  =  -4.8e9
  [31]  =  7.2e9
  [32]  =  4.8e9
12   [25]  =  -7.2e9
  [32]  =  -4.8e9
  [34]  =  7.2e9
  [35]  =  4.8e9


vy

In [32]:
figure()
subplot(1,3,1)
pcolor(grid.x,grid.y,vx)
gca().set_aspect("equal")
gca().invert_yaxis()

subplot(1,3,2)
pcolor(grid.x,grid.y,vy)
gca().set_aspect("equal")
gca().invert_yaxis()

subplot(1,3,3)
pcolor(grid.x,grid.y,P)
gca().set_aspect("equal")
gca().invert_yaxis()
show()

figure()
plot(vx[:,2])
show()

LoadError: BoundsError: attempt to access 3×4 Matrix{Float64} at index [1:3, 21]

In [6]:
vy[:,end]

82-element Vector{Float64}:
 0.0
 4.199921378911822e-10
 8.384886832964549e-10
 1.2536082905974037e-9
 1.663142230391439e-9
 2.06462579966664e-9
 2.45541958922229e-9
 2.8327968415101646e-9
 3.1940328413029557e-9
 3.5364922855216877e-9
 3.857710854350525e-9
 4.155467710160173e-9
 4.427846326082742e-9
 ⋮
 7.148060596493723e-10
 6.500813921842898e-10
 5.854159506331903e-10
 5.207463875047236e-10
 4.5602313274487125e-10
 3.9120999087261257e-10
 3.2628371960986565e-10
 2.6123361429067976e-10
 1.960611241265328e-10
 1.3077952829616152e-10
 6.54137015303397e-11
 0.0